In [1]:
from ret import read_data

In [2]:
genres = read_data("genres")

In [3]:
import numpy as np
import pandas as pd
import ast
from functools import reduce

In [4]:
retrieved_ids = ['15blZOCUg63HosU3',
 '3cMJTipuaJSlq27p',
 'oiJUeLdVwgBzhPiI',
 'eOvLEAOmwApxf5JQ',
 'vOGoMtp0LQ2fzS3F',
 'vai5vPlNfTBiu6Nj',
 'pM5Hf0ucqHSGr3jp',
 '4sUNaDw0evcjjBzv',
 'IvFtcAajHxKWNl7x',
 'i3Hq3mMS2pL8aWsU']
query_id = '01Yfj2T3YTwJ1Yfy'

In [5]:

"""
genre coverage @ 10
genres - pd.DataFrame - genre data set 
queries - nd.array - 2d array of results from query in ids 

return: 
res - float - genre coverage @ 10 score


dependency: import numpy as np 
            import pandas as pd 
"""

def gen_cov_10(retrieved, genres):
    # 1.return number of unique genre in the dataset (offline, need optimization)
    # 1.1 convert all the values in column "genre" from str to nd.array
    genres["genre_arr"] = genres["genre"].apply(lambda x: np.array(ast.literal_eval(x)))
    
    # 1.2 return the union of all genres
    all_genres = reduce(np.union1d, genres["genre_arr"])
    num_all_genres = len(all_genres)
    
    # 2.return number of unique genre in the retrieved 
    # 2.1 return genre of queries in genre with id as index 
    retrieved_df = genres.loc[genres["id"].isin(retrieved.flatten())]
    
    # 2.2 return the union of all genres in queries 
    retrieved_genres = reduce(np.union1d, retrieved_df["genre_arr"]) 
    num_retrieved_genres = len(retrieved_genres)
    
    # 3. calculate the genre coverage@10
    res = num_retrieved_genres / num_all_genres 
    return res

gen_cov_10(np.array(retrieved_ids), genres)

0.04946043165467626

In [6]:
"""
ndcg@10 score
query_id - str - query id 
retrieved_ids - List[str] - id of the retrieved tracks 
genres - pd.DataFrame - genre dataset 


return:
ndcg - float - ndcg@10 score 

"""

def ndcg_score(query_id, retrieved_ids, genres):
    # 1. convert all the values in column "genre" from str to nd.array
    genres["genre_arr"] = genres["genre"].apply(lambda x: np.array(ast.literal_eval(x)))
    
    # 2. calculate the rel for each track 
    query_genre = genres.loc[genres["id"] == query_id, 'genre_arr'].to_numpy()[0]
    retrieved_genre = pd.DataFrame(retrieved_ids, columns=['id'])
    retrieved_genre = pd.merge(genres, retrieved_genre, on="id", how="right")
    retrieved_genre["rel"] = retrieved_genre["genre_arr"].apply(lambda x: 2 * len(np.intersect1d(x, query_genre)) / (len(x) + len(query_genre)))
    
    # 3. calculate dcg
    rel = retrieved_genre["rel"].to_numpy()
    gain = np.empty(rel.shape)
    for i, _ in enumerate(rel):
        gain[...,i] = rel[...,i] / np.log2(i + 2)
    dcg = np.sum(gain)
    
    # 4. calculate idcg
    rel_sort = np.sort(rel)[::-1]
    rel_sort_gain = np.empty(rel_sort.shape)

    for i, _ in enumerate(rel_sort):
        rel_sort_gain[...,i] = rel_sort[...,i] / np.log2(i + 2)
    idcg = np.sum(rel_sort_gain)
    
    # 5. calculate ndcg
    ndcg = dcg / idcg
    return ndcg 

ndcg_score(query_id, retrieved_ids, genres)

0.6498507364439309